#### Download Stock and Option Price Data for Group of Securities.

In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

from datetime import datetime, date

import pandas_datareader.data as web
from pandas_datareader.yahoo.headers import DEFAULT_HEADERS

import requests
from time import sleep

import yfinance as yf # for minute data, https://github.com/ranaroussi/yfinance

In [2]:
# Functions to retrieve Stock Price and Options Data

# Get daily stock price data using Pandas Datareader from Yahoo! Finance
def get_stock_price_using_pdr(security, period=1):
    # security = Ticker symbol of stock, string ex: 'TQQQ'
    # period = Years of data to reterive, integer ex: 1 for 1 year
    session = requests.Session()
    session.headers = DEFAULT_HEADERS

    source = 'yahoo' # Source of data
    end_date = datetime.now()
    start_date = end_date.replace(year = end_date.year - period)
        
    df = web.DataReader(security, source, start_date, end_date, session=session)
    df = df.sort_values(by='Date')
    df = df.dropna()
    
    # Save to CSV file
    file_name = 'data/{}_daily_{}.csv'.format(security, df.index[-1].strftime('%Y%m%d'))
    df.to_csv(file_name)
    #print('{} stock {} year daily price data downloaded.'.format(security, period))
    
    return file_name

# Get minute level stock price data using yfinance
def get_stock_price_data_using_yf(security, period='1mo', interval='2m'):
    # security = Ticker symbol, string ex: 'AAPL'
    # period = Months/Days of data to retrieve, string ex: '1mo' for 1 month
    # interval = Interval between price data, string ex: '2m' for 2 minute
            
    df = yf.Ticker(security).history(period=period, interval=interval, \
                                           actions=False, auto_adjust=False)
    df = df.sort_values(by='Datetime')
    df = df.dropna()

    # Save to CSV file
    file_name = 'data/{}_{}_{}.csv'.format(security, interval, \
                                           df.index[-1].strftime('%Y%m%d%H%M%S'))
    df.to_csv(file_name)
    #print('{} stock {} month {} price data downloaded.'.format(security, period, interval))

    return file_name

# Get Option Data using pandas datareader
def get_options_data_using_pdr(security):
    # Retrieve Options Data from Yahoo! Finance
    session = requests.Session()
    session.headers = DEFAULT_HEADERS

    options = web.YahooOptions(security, session=session)
    df = options.get_all_data()

    # Flatten the option pricing df and save as CSV
    df = df.reset_index()

    # find the latest Quote time
    latest_quote = df.Quote_Time.max()
        
    # Save to CSV file
    file_name = 'data/{}_options_{}.csv'.format(security, latest_quote.strftime('%Y%m%d'))
    df.to_csv(file_name, index=False)
    #print('{} option data downloaded.'.format(security))
    
    return file_name

# Get Historical Corporate Actions using pandas datareader
def get_actions_using_pdr(security):
    # Retrieve Historical Corporate Actions from Yahoo! Finance
    actions = web.DataReader(security, 'yahoo-actions')
    
    if (actions.empty != True):
        # Save to CSV file
        file_name = 'data/{}_actions.csv'.format(security)
        actions.to_csv(file_name)
        #print('{} corporate actions data downloaded.'.format(security))

        return file_name
    else:
        return None

# Get Next Corporate Activity using yfinance
def get_calendar_using_yf(security):
    x = yf.Ticker(security)
    
    calendar = x.calendar
    
    if (calendar is not None):
        # Save to CSV file
        file_name = 'data/{}_calendar.csv'.format(security)
        calendar.to_csv(file_name)
        #print('{} calendar data downloaded'.format(security))
        
        return file_name
    else:
        return None
    
# Get Historical Earnings using yfinance
def get_earnings_hist_using_yf(security):
    x = yf.Ticker(security)
    
    earnings_hist = x.earnings_dates
    
    if (earnings_hist.empty != True):
        # Save to CSV file
        file_name = 'data/{}_earnings_hist.csv'.format(security)
        earnings_hist.to_csv(file_name)
        #print('{} earnings history data downloaded'.format(security))
    
        return file_name
    else:
        return None
    
# Get Historical Earnings and price change from Earnings from TipRanks
def get_earnings_hist_from_tipranks(security):
    
    try:
        # Get Earnings data for specific security from TipRanks

        url = 'https://www.tipranks.com/stocks/{}/earnings'
        header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.95 Safari/537.36'}

        r = requests.get(url.format(security), headers=header)


        # Eaxtract data from Earnings History table
        eps_hist_df = pd.read_html(r.content, match="EPS YoY Change", index_col=None, parse_dates=True)
        eps_hist_df = eps_hist_df[0]

        df = pd.DataFrame({
            'report_date': pd.to_datetime(eps_hist_df['Report Date']), 
            'eps_prev_yr': eps_hist_df["Last Year's EPS"]
        })

        df = df.join([
            eps_hist_df['Fiscal Quarter'].str.split(' ', expand=True).rename(columns={0: 'fiscal_yr', 1: 'fiscal_qtr'}),
            eps_hist_df['Forecast / EPS'].str.split('/', expand=True).rename(columns={0: 'eps_forecast', 1: 'eps_actual'}),
            eps_hist_df['EPS YoY Change'].str.split('% ', expand=True).rename(columns={0: 'eps_yoy_pct', 1: 'eps_yoy_chg'})
        ])

        df['fiscal_qtr'] = df['fiscal_qtr'].str[2:3]
        df['eps_yoy_chg'] = df['eps_yoy_chg'].str[1:-1]

        columns=['report_date', 'fiscal_yr', 'fiscal_qtr', 'eps_forecast', 'eps_actual', 'eps_prev_yr', 'eps_yoy_chg', 'eps_yoy_pct']

        df = df.reindex(columns=columns)

        # Extract data from Price Change table
        price_change_df = pd.read_html(r.content, match="Price 1 Day Before", index_col=None, parse_dates=True)
        price_change_df = price_change_df[0]

        df1 = pd.DataFrame({
            'report_date': pd.to_datetime(price_change_df['Report Date']), 
            'price_1d_before': price_change_df['Price 1 Day Before'].str[1:], 
            'price_1d_after': price_change_df['Price 1 Day After'].str[1:],
            'price_pct_change': price_change_df['Percentage Change'].str[:-1]
        })

        # Merge past earnings df and price change due to earnings df
        df2 = df.merge(df1, on='report_date', how='left')

        # Change columns data type
        cols = df2.select_dtypes(include=['object']).columns
        df2[cols] = df2[cols].apply(pd.to_numeric, errors='coerce')
        #display(df2)
    except:
        df2 = None
    
    if (df2.empty != True):
        # Save to CSV file
        file_name = 'data/{}_earnings_hist_tipranks.csv'.format(security)
        df2.to_csv(file_name)
        #print('{} TipRanks earnings history data downloaded'.format(security))
    
        return file_name
    else:
        return None

In [3]:
# Group of Securities

# portfolio holdings
etf_holdings = ['TQQQ', 'UPRO']
stock_holdings = ['META', 'AAPL', 'PLTR', 'GPRO', 'C']

# trades executed with year
etf_trades = ['TLT', 'SOXL', 'SQQQ', 'DIA', 'USO']
stock_trades = ['DOW', 'NKE', 'WMT', 'TWTR', 'MRK', 'XOM', 'PYPL', 'AMC', 'RBLX', 'DIS', 'SE', 'HD', \
                'M', 'PANW', 'ZM', 'SNOW', 'CRM', 'NVDA', 'CHPT', 'OKTA', 'BTU', 'MSFT', 'AMZN', 'GOOGL', \
                'GME', 'AEO', 'DOCU', 'KR', 'PINS', 'NIO', 'ABNB', 'DVN', 'ADP', 'AXP', 'BX', 'CGC', 'CLOV', \
                'CMCSA', 'LMT', 'RKT', 'SOFI', 'UNH', 'BIIB', 'CI', 'COST', 'TSLA']

# index etfs for tracking
index_etfs = ['SPY', 'QQQ', 'IWM', 'LQD', 'UVXY', 'VXX']

# new etfs and stocks under consideration
new_etfs = ['XBI', 'XLK', 'XLY', 'XLF', 'XLE', 'XOP', 'ARKK', 'TBT', 'ARKG', 'SMH']
new_stocks = ['BABA', 'INTC', 'AMD', 'T', 'V', 'SBUX', 'UBER']

# potential earnings play
earning_stocks = ['RAD', 'NKE', 'MU', 'CCL', 'STZ', 'CAG', 'LEVI', 'AEHR', 'TLRY']

# etf_list is used to filter earnings report from TipRanks
etf_list = etf_holdings + etf_trades + index_etfs + new_etfs

master_list = etf_holdings + stock_holdings + etf_trades + stock_trades + index_etfs + new_stocks + new_etfs + earning_stocks

# remove duplicates while keeping order
master_list = sorted(set(master_list), key=master_list.index)

# Exclude tickers (low open interest, low price)
#excluded_tickers = ['ADP', 'LMT', 'UNH']
excluded_tickers = []

securities = [ticker for ticker in master_list if ticker not in excluded_tickers]

# PmTraders lotto (Not included in primary download; download separately)
lotto_stocks = ['AA', 'ABC', 'ABNB', 'ABT', 'ACHC', 'ACN', 'ADBE', 'ADP', 'ADSK', 'AFL', 'ALB', 'ALGN', 'AMBA', 'AMD', 'AMG', 'AMGN', 'ANET', 'APA', 'APO', 'ASML', \
                'AXON', 'AXP', 'AZO', 'BIIB', 'BILL', 'BLK', 'BNTX', 'BUD', 'BURL', 'BX', 'CAT', 'CCI', 'CF', 'CHTR', \
                'CI', 'CLX', 'CMG', 'COUP', 'CTAS', 'CTRA', 'CVNA', 'CZR', 'DE', 'DFS', 'DHI', 'DIOD', 'DKS', 'DOCU', 'DPZ', 'DUOL', 'EL', \
                'ENPH', 'EOG', 'EXPE', 'EW', 'FCX', 'FFIV', 'FIVE', 'FSLR', 'GD', 'GFS', 'GS', \
                'HES', 'HP', 'HSY', 'HUBB', 'HUBS', 'ILMN', 'INTU', 'IR', 'ISRG', 'ITW', 'JBHT', \
                'KKR', 'KLAC', 'LEN', 'LHX', 'LIN', 'LLY', 'LMT', 'LNG', 'MA', 'MAR', 'MCK', 'MDB', 'MELI', 'MOS', \
                'MPC', 'NDSN', 'NET', 'NFLX', 'NOW', 'NSC', 'NTR', 'OIH', 'OKTA', 'OLED', 'OLN', 'ON', 'PANW', 'PEP', 'PM', \
                'PNC', 'PPG', 'PSX', 'PXD', 'RACE', 'RCL', 'REGN', 'RH', 'ROKU', 'ROST', 'RRC', 'SLB', 'SNOW', 'SPGI', 'SPLK', 'SPOT', \
                'SPT', 'SQM', 'SWKS', 'SYF', 'SYK', 'TEAM', 'TELL', 'TM', 'TMO', 'TNDM', 'TOL', \
                'TSCO', 'TTD', 'TTWO', 'TWLO', 'U', 'UFCS', 'ULTA', 'UNP', 'URI', 'USO', 'VAC', 'VFC', 'VLO', 'VRTX', \
                'W', 'WDAY', 'WDC', 'WHR', 'WM', 'WYNN', 'XHB', 'ZBH', 'ZS']

# debug
#securities = set(['NG', 'STZ', 'CAG', 'LEVI', 'AEHR', 'TLRY'])
#securities = set(earning_stocks)
# securities = set(lotto_stocks)

# new tickers that are not in previous lists for download
#master_list = set(master_list + lotto_stocks)
#securities = set([ticker for ticker in lotto_stocks if ticker not in master_list])

In [4]:
data = []
downloaded_tickers = []
troubled_tickers = []

# [NOTE]: Randomizing list wasn't effective.
# Yahoo! seems to send incomplete data after a while.
# Put important tickers first.
#shuffle(securities)

print('Data for {} securities to be downloaded'.format(len(securities)))

pbr = tqdm(securities)
for security in pbr:
    # The download for this section must be clean and complete
    # Redownload incomplete downloaded securities
    try:
        # daily price data
        data.append([security, 'daily', get_stock_price_using_pdr(security)])

        # 2 minute price data
        data.append([security, '2min', get_stock_price_data_using_yf(security, period='1mo', interval='2m')])

        # Option data
        data.append([security, 'option', get_options_data_using_pdr(security)])

        # Add to list of downloded tickers
        downloaded_tickers.append(security)

        # sleep randomly
        sleep(np.random.uniform(1,2))
    except:
        print('Download incomplete for: {}'.format(security))
        troubled_tickers.append(security)
    
    pbr.set_description('Downloading Price Data for {}'.format(security))
    
# Successful and incomplete download info
print('Incomplete for {}: {}'.format(len(troubled_tickers), troubled_tickers))
print('Successful for {}: {}'.format(len(downloaded_tickers), downloaded_tickers))

Data for 92 securities to be downloaded


Incomplete for 0: []
Successful for 92: ['TQQQ', 'UPRO', 'META', 'AAPL', 'PLTR', 'GPRO', 'C', 'TLT', 'SOXL', 'SQQQ', 'DIA', 'USO', 'DOW', 'NKE', 'WMT', 'TWTR', 'MRK', 'XOM', 'PYPL', 'AMC', 'RBLX', 'DIS', 'SE', 'HD', 'M', 'PANW', 'ZM', 'SNOW', 'CRM', 'NVDA', 'CHPT', 'OKTA', 'BTU', 'MSFT', 'AMZN', 'GOOGL', 'GME', 'AEO', 'DOCU', 'KR', 'PINS', 'NIO', 'ABNB', 'DVN', 'ADP', 'AXP', 'BX', 'CGC', 'CLOV', 'CMCSA', 'LMT', 'RKT', 'SOFI', 'UNH', 'BIIB', 'CI', 'COST', 'TSLA', 'SPY', 'QQQ', 'IWM', 'LQD', 'UVXY', 'VXX', 'BABA', 'INTC', 'AMD', 'T', 'V', 'SBUX', 'UBER', 'XBI', 'XLK', 'XLY', 'XLF', 'XLE', 'XOP', 'ARKK', 'TBT', 'ARKG', 'SMH', 'ADBE', 'FDX', 'RAD', 'MU', 'CCL', 'NG', 'STZ', 'CAG', 'LEVI', 'AEHR', 'TLRY']


In [5]:
# Download Tipranks earnings data separately
data = []
troubled_tickers = []

pbar = tqdm(securities)
for security in pbar:
    try:
        if security not in etf_list:
            # TipRanks Earnings history
            data.append([security, 'tipranks_earning_hist', get_earnings_hist_from_tipranks(security)])
    except:
        troubled_tickers.append(security)
        pass
    
    pbar.set_description('Downloading Tipranks Earnings for {}'.format(security))

print('Incomplete for {}: {}'.format(len(troubled_tickers), troubled_tickers))

Incomplete for 0: []


In [6]:
# Redownload of incomplete downloads
#securities = ['PYPL', 'AMC', 'RBLX', 'ABNB', 'ADP', 'AXP', 'BTU', 'BX', 'CGC', 'CLOV', 'CMCSA', 'DIS', 'UNH', 'MSFT', 'AMZN', 'GOOGL', 'BABA', 'TSLA', 'COIN', 'TTD', 'U', 'SONO', 'RIVN', 'ILMN', 'LZ', 'MTTR', 'MQ', 'PLUG', 'WYNN', 'WISH', 'WIX', 'JACK', 'UPST', 'APPS', 'TTWO', 'MARA', 'BLNK', 'LMND', 'SDC', 'DOCN', 'VRM', 'PUBM', 'LAZR', 'ROOT', 'GRPN', 'SWAV', 'RL', 'CPRI', 'WKHS', 'RMBL', 'PLBY', 'ANGI']

# Download Yahoo earnings history data separately
data = []
troubled_tickers = []

pbar = tqdm(securities)
for security in pbar:

    try:
        if security not in etf_list:
            # Earnings history
            data.append([security, 'earning_hist', get_earnings_hist_using_yf(security)])

    except:
        troubled_tickers.append(security)
        pass
    
    pbar.set_description('Downloading Earnings for {}'.format(security))

print('Incomplete for {}: {}'.format(len(troubled_tickers), troubled_tickers))

Incomplete for 44: ['DIS', 'SE', 'HD', 'M', 'PANW', 'ZM', 'SNOW', 'CRM', 'NVDA', 'CHPT', 'OKTA', 'BTU', 'MSFT', 'PINS', 'NIO', 'ABNB', 'DVN', 'ADP', 'AXP', 'BX', 'CGC', 'CLOV', 'CMCSA', 'LMT', 'RKT', 'TSLA', 'BABA', 'INTC', 'AMD', 'T', 'V', 'SBUX', 'UBER', 'ADBE', 'FDX', 'RAD', 'MU', 'CCL', 'NG', 'STZ', 'CAG', 'LEVI', 'AEHR', 'TLRY']


In [7]:
# Corporate Action data download
data = []
troubled_tickers = []

# progress bar pbar
pbar = tqdm(securities)
for security in pbar:

    try:
        # Corporate Action data
        data.append([security, 'corp_action', get_actions_using_pdr(security)])
                        
    except:
        troubled_tickers.append(security)
        pass

    pbar.set_description('Downloading Corporate Action data for {}'.format(security))

print('Incomplete for {}: {}'.format(len(troubled_tickers), troubled_tickers))

Incomplete for 0: []


In [8]:
# Download Yahoo earnings calendar data
data = []
troubled_tickers = []

# progress bar pbar
pbar = tqdm(securities)
for security in pbar:
    
    try:
        if security not in etf_list:
            # Calendar data
            data.append([security, 'calendar', get_calendar_using_yf(security)])

    except:
        troubled_tickers.append(security)
        pass
        
    pbar.set_description('Downloading Calendar data for {}'.format(security))

print('Incomplete for {}: {}'.format(len(troubled_tickers), troubled_tickers))

Incomplete for 0: []
